In [1]:
import pandas as pd
import os
import os.path
from os.path import join
import numpy as np
DATA_DIR_INITIAL = '../data/raw/'
DATA_DIR = join(DATA_DIR_INITIAL, '2nd Pass Analysis')

### Analyze categorized tables + annotations

In [2]:
files_dict = {}
files_succ = []
files_fail = []
annotators = [x for x in os.listdir(DATA_DIR) if not '.' in x]
for annotator in annotators:
    print(annotator)
    annotator_dir = join(DATA_DIR, annotator)
    interviews = [x for x in os.listdir(
        annotator_dir) if x.endswith('.xlsx') and not 'INCOMPLETE' in x]
    # files_dict[annotator] = interviews
    for interview in interviews:
        fname = join(annotator_dir, interview)
        df = pd.read_excel(fname)
        key = interview.replace('Interview Analysis.xlsx', '').strip()
        files_dict[key] = df
        COLS = ['Item #', 'Slide #', 'Domain', 'Subcategory']
        for i in range(4):
            assert df.columns[i] == COLS[i]
        assert df.shape[0] == 46
        # print(df.shape)
        # assert df.columns[0] == 'Item #'
        # assert df.columns[1] == 'Slide #'
        # display(df.head())
        # print(df.columns)
        try:
            # , interview + ' ' + df.columns
            assert 'theme' in pd.Series(df.columns).apply(str.lower)
            theme_index = np.where(
                np.array(list(map(str.lower, df.columns.values))) == 'theme')[0][0]
            for i in range(theme_index + 1, len(df.columns), 2):
                assert '#' in df.columns[i]
            assert df.shape[0] == 46
            files_succ.append(key)
        except:
            files_fail.append(key)
print('success:', len(files_succ))
print('fail:', files_fail)

Kevin_AnnArbor_LosAngeles_SaltLakeCity
Jonathan_Indianapolis_Milwaukee_Pittsburgh
Katie_Dallas_Seattle_Tucson


/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Spencer_Denver_Sacramento
Patty_Charlotte_Houston_Portland
Corina_Buffalo_Cincinnati_SanFrancisco
Manish_Atlanta_Columbus_WashingtonDC
success: 0
fail: ['Los Angeles COMPLETE', 'Salt Lake City', 'Ann Arbor', 'Pittsburgh', 'Milwaukee', 'Indianapolis', 'Tucson', 'Dallas', 'Seattle', 'Denver', 'Sacramento', 'Houston', 'Charlotte', 'Portland', 'Cincinnati', 'San Francisco', 'Buffalo', 'Columbus', 'WashingtonDC', 'Atlanta']


/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [8]:
# for k in sorted(files_succ):
#     df = files_dict[k]
k = 'Atlanta'
df = files_dict[key]

In [13]:
theme_index = np.where(
    np.array(list(map(str.lower, df.columns.values))) == 'theme')[0][0]
col_vals = df.columns[4: theme_index]

# separate into relevant pieces
qs = df['Subcategory']
responses_df = df[col_vals]
themes_df = df[df.columns[theme_index:]]

In [23]:
def get_interview_set(question_num: int, qs, responses_df, themes_df):
    question = qs.iloc[question_num]
    responses = responses_df.iloc[question_num]
    theme_row = themes_df.iloc[question_num].values
    theme_dict = {theme_row[i]: theme_row[i + 1]
                  for i in range(0, len(theme_row), 2)}

    for k in list(theme_dict.keys()):
        # remove nan/whitespace from theme_dict
        if type(k) != str or not k.strip():
            del theme_dict[k]
    return question, responses, theme_dict


# theme_row = get_interview_set(0, qs, responses, themes)
question, responses, theme_dict = get_interview_set(
    6, qs, responses_df, themes_df)

In [24]:
question

'In that moment, when a paramedic is getting ready to treat your child who is having a seizure, would you have been able to make an informed decision about whether or not to participate in a research study? '

In [25]:
responses.values

array(['Yes, I just want him treated', 'I would want them to just do it',
       'Yes', 'Yes', 'Yes', "It wouldn't be the right time to ask", 'Yes',
       'No', 'No', 'Yes', nan], dtype=object)

In [26]:
theme_dict

{'Yes': 7, 'No': 2, "Wouldn't be the right time to ask": 1}

### Analyze responses

In [27]:
import guidance